# sprint14 ディープラーニング フレームワーク2

## 1.このSprintについて

**Sprintの目的**

- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

**どのように学ぶか**

前半はTensorFlowのExampleを動かします。後半ではKerasのコードを書いていきます。

## 2.公式Example

深層学習フレームワークには公式に様々なモデルのExampleコードが公開されています。

## 【問題1】公式チュートリアルモデルを分担して実行
TensorFLowの公式チュートリアルモデルを分担して実行してください。


以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。


[models/tutorials at master · tensorflow/models](https://www.tensorflow.org/tutorials/)


**選んだチュートリアルモデル：[RNN によるテキスト生成](https://www.tensorflow.org/tutorials/text/text_generation)**

**RNNとは**
時系列を扱うニューラルネットの一種。自然言語などの処理に使われる


> このチュートリアルでは、文字ベースの RNN を使ってテキストを生成する方法を示します。ここでは、Andrej Karpathy の [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) からのシェイクスピア作品のデータセットを使います。このデータからの文字列（"Shakespear"）を入力にして、文字列中の次の文字（"e"）を予測するモデルを訓練します。このモデルを繰り返し呼び出すことで、より長い文字列を生成することができます。

このチュートリアルには、tf.keras と eager execution を使ったコードが含まれています。下記は、このチュートリアルのモデルを 30 エポック訓練したものに対して、文字列 "Q" を初期値とした場合の出力例です。

いくつかは文法にあったものがある一方で、ほとんどは意味をなしていません。このモデルは、単語の意味を学習していませんが、次のことを考えてみてください。

- このモデルは文字ベースです。訓練が始まった時に、モデルは英語の単語のスペルも知りませんし、単語がテキストの単位であることも知らないのです。

- 出力の構造は戯曲に似ています。だいたいのばあい、データセットとおなじ大文字で書かれた話し手の名前で始まっています。

- 以下に示すように、モデルはテキストの小さなバッチ（各100文字）で訓練されていますが、一貫した構造のより長いテキストのシーケンスを生成できます。

## 設定
### TensorFlow 等のライブラリインポート

In [1]:
# インポート
import tensorflow as tf

import numpy as np
import os
import time

### シェイクスピアデータセットのダウンロード

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### データの読み込み

In [3]:
# 読み込んだのち、Python 2 との互換性のためにデコード
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# テキストの長さは含まれる文字数
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# テキストの最初の 250文字を参照
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# ファイル中のユニークな文字の数
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
print(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## テキストの処理
### テキストのベクトル化
訓練をする前に、文字列を数値表現に変換する必要があります。2つの参照テーブルを作成します。一つは文字を数字に変換するもの、もう一つは数字を文字に変換するものです。

In [7]:
# それぞれの文字からインデックスへの対応表を作成
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

これで、それぞれの文字を整数で表現できました。文字を、0 からlen(unique) までのインデックスに変換していることに注意してください。

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
# テキストの最初の 13 文字がどのように整数に変換されるかを見てみる
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### 予測タスク

**ある文字、あるいは文字列が与えられたとき、もっともありそうな次の文字はなにか？これが、モデルを訓練してやらせたいタスク** です。モデルへの入力は文字列であり、モデルが出力、つまりそれぞれの時点での次の文字を予測をするようにモデルを訓練します。

RNN はすでに見た要素に基づく内部状態を保持しているため、この時点までに計算されたすべての文字を考えると、次の文字は何でしょうか？

### 訓練用サンプルとターゲットを作成

つぎに、テキストをサンプルシーケンスに分割します。それぞれの入力シーケンスは、元のテキストからの seq_length 個の文字を含みます。

入力シーケンスそれぞれに対して、対応するターゲットは同じ長さのテキストを含みますが、1文字ずつ右にシフトしたものです。

そのため、テキストを seq_length+1 のかたまりに分割します。たとえば、 seq_length が 4 で、テキストが "Hello" だとします。入力シーケンスは "Hell" で、ターゲットシーケンスは "ello" となります。

これを行うために、最初に [tf.data.Dataset.from_tensor_slices](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) 関数を使ってテキストベクトルを文字インデックスの連続に変換します。

In [10]:
# ひとつの入力としたいシーケンスの文字数としての最大の長さ
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# 訓練用サンプルとターゲットを作る
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [11]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [12]:
text_as_int.shape

(1115394,)

batch メソッドを使うと、個々の文字を求める長さのシーケンスに簡単に変換できます。

In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


シーケンスそれぞれに対して、map メソッドを使って各バッチに単純な関数を適用することで、複製とシフトを行い、入力テキストとターゲットテキストを生成します。

In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

最初のサンプルの入力とターゲットを出力します。

In [15]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


これらのベクトルのインデックスそれぞれが一つのタイムステップとして処理されます。タイムステップ 0 の入力として、モデルは "F" のインデックスを受け取り、次の文字として "i" のインデックスを予測しようとします。次のタイムステップでもおなじことをしますが、**RNN は現在の入力文字に加えて、過去のステップのコンテキストも考慮** します。

In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### 訓練用バッチの作成
[tf.data](https://www.tensorflow.org/api_docs/python/tf/data) を使ってテキストを分割し、扱いやすいシーケンスにします。しかし、このデータをモデルに供給する前に、データをシャッフルしてバッチにまとめる必要があります。

In [17]:
# バッチサイズ
BATCH_SIZE = 64

# データセットをシャッフルするためのバッファサイズ
# （TF data は可能性として無限長のシーケンスでも使えるように設計されています。
# このため、シーケンス全体をメモリ内でシャッフルしようとはしません。
# その代わりに、要素をシャッフルするためのバッファを保持しています）
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### モデルの構築

[tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) を使ってモデルを定義します。この簡単な例では、モデルの定義に3つのレイヤーを使用しています。

- [tf.keras.layers.Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding): 入力レイヤー。それぞれの文字を表す数を embedding_dim　次元のベクトルに変換する、訓練可能な参照テーブル。
- [tf.keras.layers.GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU): サイズが units=rnn_units のRNNの一種（ここに LSTM レイヤーを使うこともできる）。
- [tf.keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): vocab_size の出力を持つ、出力レイヤー。

In [18]:
# 文字数で表されるボキャブラリーの長さ
vocab_size = len(vocab)

# 埋め込みベクトルの次元
embedding_dim = 256

# RNN ユニットの数
rnn_units = 1024

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [20]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

1文字ごとにモデルは埋め込みベクトルを検索し、その埋め込みベクトルを入力として GRU を 1 タイムステップ実行します。そして Dense レイヤーを適用して、次の文字の対数尤度を予測するロジットを生成します。

## モデルを試す
期待通りに動作するかどうかを確認するためモデルを動かしてみましょう。

最初に、出力の shape を確認します。

**※バージョンの問題か、build_model関数を呼び出して動かすたびに、Jupyterカーネルが落ちてしまう。⇒一旦コメントアウトし、Google Colabで試した結果を載せる。**

In [21]:
# for input_example_batch, target_example_batch in dataset.take(1):
#   example_batch_predictions = model(input_example_batch)
#   print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [22]:
# (64, 100, 65) # (batch_size, sequence_length, vocab_size)

上記の例では、入力のシーケンスの長さは 100 ですが、モデルはどのような長さの入力でも実行できます。

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


モデルから実際の予測を得るには出力の分布からサンプリングを行う必要があります。この分布は、文字ボキャブラリー全体のロジットで定義されます。

> Note: この分布から サンプリング するということが重要です。なぜなら、分布の argmax をとったのでは、モデルは簡単にループしてしまうからです。

バッチ中の最初のサンプルで試してみましょう。

In [24]:
# sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

これにより、タイムステップそれぞれにおいて、次の文字のインデックスの予測が得られます。

In [25]:
# sampled_indices

これらをデコードすることで、この訓練前のモデルによる予測テキストをみることができます。

In [26]:
# print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
# print()
# print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

## モデルの訓練

ここまでくれば問題は標準的な分類問題として扱うことができます。これまでの RNN の状態と、いまのタイムステップの入力が与えられ、次の文字のクラスを予測します。

### オプティマイザと損失関数の付加
この場合、標準の [tf.keras.losses.sparse_categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/sparse_categorical_crossentropy) 損失関数が使えます。予測の最後の次元に適用されるからです。

このモデルはロジットを返すので、from_logits フラグをセットする必要があります。

In [27]:
# def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# example_batch_loss  = loss(target_example_batch, example_batch_predictions)
# print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
# print("scalar_loss:      ", example_batch_loss.numpy().mean())

[tf.keras.Model.compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) を使って、訓練手順を定義します。
既定の引数を持った [tf.keras.optimizers.Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) と、先ほどの loss 関数を使用しましょう。

In [28]:
# model.compile(optimizer='adam', loss=loss)

## チェックポイントの構成
[tf.keras.callbacks.ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) を使って、訓練中にチェックポイントを保存するようにします。

In [29]:
# チェックポイントが保存されるディレクトリ
checkpoint_dir = './training_checkpoints'
# チェックポイントファイルの名称
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## 訓練の実行
訓練時間を適切に保つために、10エポックを使用してモデルを訓練します。Google Colab を使用する場合には、訓練を高速化するためにランタイムを GPU に設定します。

In [41]:
# EPOCHS=10

In [31]:
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

## テキスト生成

### 最終チェックポイントの復元
予測ステップを単純にするため、バッチサイズ 1 を使用します。

RNN が状態をタイムステップからタイムステップへと渡す仕組みのため、モデルは一度構築されると固定されたバッチサイズしか受け付けられません。

モデルを異なる batch_size で実行するためには、モデルを再構築し、チェックポイントから重みを復元する必要があります。

In [32]:
# tf.train.latest_checkpoint(checkpoint_dir)

In [33]:
# model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# model.build(tf.TensorShape([1, None]))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## 予測ループ
下記のコードブロックでテキストを生成します。

- 最初に、開始文字列を選択し、RNN の状態を初期化して、生成する文字数を設定します。

- 開始文字列と RNN の状態を使って、次の文字の予測分布を得ます。

- つぎに、カテゴリー分布を使用して、予測された文字のインデックスを計算します。この予測された文字をモデルの次の入力にします。

- モデルによって返された RNN の状態はモデルにフィードバックされるため、1つの文字だけでなく、より多くのコンテキストを持つことになります。つぎの文字を予測した後、更新された RNN の状態が再びモデルにフィードバックされます。こうしてモデルは以前に予測した文字からさらにコンテキストを得ることで学習するのです。

To generate text the model's output is fed back to the input

生成されたテキストを見ると、モデルがどこを大文字にするかや、段落の区切り方、シェークスピアらしい書き言葉を真似ることを知っていることがわかります。しかし、訓練のエポック数が少ないので、まだ一貫した文章を生成するところまでは学習していません。

In [35]:
# def generate_text(model, start_string):
#   # 評価ステップ（学習済みモデルを使ったテキスト生成）

#   # 生成する文字数
#   num_generate = 1000

#   # 開始文字列を数値に変換（ベクトル化）
#   input_eval = [char2idx[s] for s in start_string]
#   input_eval = tf.expand_dims(input_eval, 0)

#   # 結果を保存する空文字列
#   text_generated = []

#   # 低い temperature　は、より予測しやすいテキストをもたらし
#   # 高い temperature は、より意外なテキストをもたらす
#   # 実験により最適な設定を見つけること
#   temperature = 1.0

#   # ここではバッチサイズ　== 1
#   model.reset_states()
#   for i in range(num_generate):
#       predictions = model(input_eval)
#       # バッチの次元を削除
#       predictions = tf.squeeze(predictions, 0)

#       # カテゴリー分布をつかってモデルから返された文字を予測 
#       predictions = predictions / temperature
#       predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

#       # 過去の隠れ状態とともに予測された文字をモデルへのつぎの入力として渡す
#       input_eval = tf.expand_dims([predicted_id], 0)

#       text_generated.append(idx2char[predicted_id])

#   return (start_string + ''.join(text_generated))

In [36]:
# print(generate_text(model, start_string=u"ROMEO: "))

**この結果を改善するもっとも簡単な方法は、もっと長く訓練することです（EPOCHS=30 を試してみましょう）。**

**また、異なる初期文字列を使ったり、モデルの精度を向上させるためにもうひとつ RNN レイヤーを加えたり、temperature パラメータを調整して、よりランダム性の強い、あるいは、弱い予測を試してみたりすることができます。**

## 上級編： 訓練のカスタマイズ
上記の訓練手順は単純ですが、制御できるところがそれほどありません。

モデルを手動で実行する方法を見てきたので、訓練ループを展開し、自分で実装してみましょう。このことが、たとえばモデルのオープンループによる出力を安定化するための カリキュラム学習 を実装するための出発点になります。

勾配を追跡するために [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) を使用します。このアプローチについての詳細を学ぶには、 [eager execution guide](https://www.tensorflow.org/guide/eager) をお読みください。

この手順は下記のように動作します。

- 最初に、RNN の状態を初期化する。[tf.keras.Model.reset_states](https://www.tensorflow.org/api_docs/python/tf/keras/Model#reset_states) メソッドを呼び出すことでこれを実行する。

- つぎに、（1バッチずつ）データセットを順番に処理し、それぞれのバッチに対する予測値を計算する。

- [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) をオープンし、そのコンテキストで、予測値と損失を計算する。

- tf.GradientTape.grads メソッドを使って、モデルの変数に対する損失の勾配を計算する。

- 最後に、オプティマイザの tf.train.Optimizer.apply_gradients メソッドを使って、逆方向の処理を行う。

In [37]:
# model = build_model(
#   vocab_size = len(vocab),
#   embedding_dim=embedding_dim,
#   rnn_units=rnn_units,
#   batch_size=BATCH_SIZE)

In [38]:
# optimizer = tf.keras.optimizers.Adam()

In [39]:
# @tf.function
# def train_step(inp, target):
#   with tf.GradientTape() as tape:
#     predictions = model(inp)
#     loss = tf.reduce_mean(
#         tf.keras.losses.sparse_categorical_crossentropy(
#             target, predictions, from_logits=True))
#   grads = tape.gradient(loss, model.trainable_variables)
#   optimizer.apply_gradients(zip(grads, model.trainable_variables))

#   return loss

In [40]:
# # 訓練ステップ
# EPOCHS = 10

# for epoch in range(EPOCHS):
#   start = time.time()

#   # 各エポックの最初に、隠れ状態を初期化する
#   # 最初は隠れ状態は None
#   hidden = model.reset_states()

#   for (batch_n, (inp, target)) in enumerate(dataset):
#     loss = train_step(inp, target)

#     if batch_n % 100 == 0:
#       template = 'Epoch {} Batch {} Loss {}'
#       print(template.format(epoch+1, batch_n, loss))

#   # 5エポックごとにモデル（のチェックポイント）を保存する
#   if (epoch + 1) % 5 == 0:
#     model.save_weights(checkpoint_prefix.format(epoch=epoch))

#   print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
#   print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# model.save_weights(checkpoint_prefix.format(epoch=epoch))

【問題2】（アドバンス課題）様々な手法を実行
TensorFLowやGoogle AI ResearchのGitHubリポジトリには、定番のモデルから最新のモデルまで多様なコードが公開されています。これらから興味あるものを選び実行してください。


なお、これらのコードは初学者向けではないため、巨大なデータセットのダウンロードが必要な場合など、実行が簡単ではないこともあります。そういった場合は、コードリーディングを行ってください。


models/research at master · tensorflow/models


google-research/google-research: Google AI Research


更新日が古いものはPythonやTensorFlowのバージョンが古く、扱いずらい場合があります。新しいものから見ることを推奨します。

## 3.異なるフレームワークへの書き換え

「ディープラーニングフレームワーク1」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。


- Iris（Iris-versicolorとIris-virginicaのみの2値分類）
- Iris（3種類全ての目的変数を使用して多値分類）
- House Prices
- MNIST


### Kerasへの書き換え
KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。


KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。

## 【問題3】Iris（2値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as K 

K.clear_session()

In [50]:
# データセットの読み込み
df = pd.read_csv("Iris.csv")

In [51]:
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

In [52]:
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

In [53]:
# ラベルを数値に変換
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

In [54]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print('X_train.shape', X_train.shape)
print('X_val.shape', X_val.shape)
print('y_train', y_train.shape)
print('y_val.shape', y_val.shape)

X_train.shape (64, 4)
X_val.shape (16, 4)
y_train (64, 1)
y_val.shape (16, 1)


In [55]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [56]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 30

# モデル構築
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(4,)))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               640       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 8,961
Trainable params: 8,961
Non-trainable params: 0
_________________________________________________________________


In [57]:
# モデルコンパイル
model.compile(loss='binary_crossentropy',
             optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
             metrics=['accuracy'])

# 学習
history = model.fit(X_train, y_train,
                   batch_size=batch_size,
                   epochs=num_epochs,
                   verbose=1,
                   validation_data=(X_val,y_val))

# 推定結果(確率)
y_pred_proba = model.predict(X_train)[:,0]
# 確率を0, 1に変換
y_pred = np.where(y_pred_proba > 0.5, 1, 0)

# 指標値計算
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 64 samples, validate on 16 samples
Epoch 1/30
64/64 [==============================] - 1s 23ms/sample - loss: 0.6726 - accuracy: 0.5312 - val_loss: 0.6508 - val_accuracy: 0.7500
Epoch 2/30
64/64 [==============================] - 0s 1ms/sample - loss: 0.6573 - accuracy: 0.6250 - val_loss: 0.6172 - val_accuracy: 0.8125
Epoch 3/30
64/64 [==============================] - 0s 1ms/sample - loss: 0.6204 - accuracy: 0.7812 - val_loss: 0.6719 - val_accuracy: 0.3750
Epoch 4/30
64/64 [==============================] - 0s 1ms/sample - loss: 0.6106 - accuracy: 0.5469 - val_loss: 0.6418 - val_accuracy: 0.4375
Epoch 5/30
64/64 [==============================] - 0s 1ms/sample - loss: 0.6015 - accuracy: 0.8125 - val_loss: 0.5702 - val_accuracy: 0.8750
Epoch 6/30
64/64 [==============================] - 0s 1ms/sample - loss: 0.5768 - accuracy: 0.8906 - val_loss: 0.6064 - val_accuracy: 0.5625
Epoch 7/30
64/64 [==============================] - 0s 1ms/sample - loss: 0.5542 - accuracy: 0.7656 - v

## 【問題4】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [66]:
from sklearn.preprocessing import OneHotEncoder

In [86]:
# データセットの読み込み
df = pd.read_csv("Iris.csv")

# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 配列に変換
y = np.array(y)
X = np.array(X)

# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
# y = y.astype(np.int64)[:, np.newaxis]
y.shape

(150,)

In [87]:
# ワンホット処理
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])
y_one_hot.shape

(150, 3)

In [88]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print('X_train.shape', X_train.shape)
print('X_val.shape', X_val.shape)
print('y_train', y_train.shape)
print('y_val.shape', y_val.shape)

X_train.shape (96, 4)
X_val.shape (24, 4)
y_train (96, 3)
y_val.shape (24, 3)


In [89]:
K.clear_session()

In [90]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 30

# モデル構築
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(4,)))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               640       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 9,091
Trainable params: 9,091
Non-trainable params: 0
_________________________________________________________________


In [91]:
# モデルコンパイル
model.compile(loss='categorical_crossentropy',
             optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
             metrics=['accuracy'])

# 学習
history = model.fit(X_train, y_train,
                   batch_size=batch_size,
                   epochs=num_epochs,
                   verbose=1,
                   validation_data=(X_val,y_val))

# 推定結果(確率)
y_pred_proba = model.predict(X_train)
# 確率を0, 1に変換
y_pred = np.argmax(y_pred_proba, axis=1)

# 指標値計算
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 96 samples, validate on 24 samples
Epoch 1/30
96/96 [==============================] - 1s 12ms/sample - loss: 1.1344 - accuracy: 0.5417 - val_loss: 0.8059 - val_accuracy: 0.7083
Epoch 2/30
96/96 [==============================] - 0s 930us/sample - loss: 0.7683 - accuracy: 0.8542 - val_loss: 0.6991 - val_accuracy: 0.9167
Epoch 3/30
96/96 [==============================] - 0s 935us/sample - loss: 0.6330 - accuracy: 0.7500 - val_loss: 0.6082 - val_accuracy: 0.7083
Epoch 4/30
96/96 [==============================] - 0s 934us/sample - loss: 0.5442 - accuracy: 0.7396 - val_loss: 0.5288 - val_accuracy: 0.8333
Epoch 5/30
96/96 [==============================] - 0s 979us/sample - loss: 0.4702 - accuracy: 0.9167 - val_loss: 0.4696 - val_accuracy: 0.7500
Epoch 6/30
96/96 [==============================] - 0s 943us/sample - loss: 0.4124 - accuracy: 0.9062 - val_loss: 0.4310 - val_accuracy: 0.9167
Epoch 7/30
96/96 [==============================] - 0s 925us/sample - loss: 0.3737 - accuracy

## 【問題5】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [115]:
# データセットの読み込み
df = pd.read_csv('train.csv')
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]

# NumPy 配列に変換
y = np.array(y)
X = np.array(X)
y = y[:, np.newaxis] # (n_samples, 1)
y.shape

(1460, 1)

In [116]:
# yに対して対数変換を行う

y = np.log(y)
y

array([[12.24769432],
       [12.10901093],
       [12.31716669],
       ...,
       [12.49312952],
       [11.86446223],
       [11.90158345]])

In [117]:
# 標準化を行う
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

In [118]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(934, 2)
(234, 2)
(934, 1)
(234, 1)


In [122]:
K.clear_session()

In [123]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 30

# モデル構築
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(X.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               384       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


In [124]:
# モデルコンパイル
model.compile(loss='mean_squared_error',
             optimizer=tf.optimizers.Adam(learning_rate=learning_rate))

# 学習
history = model.fit(X_train, y_train,
                   batch_size=batch_size,
                   epochs=num_epochs,
                   verbose=1,
                   validation_data=(X_val,y_val))

# 推定結果
y_pred = model.predict(X_train)

# 指標値計算
score = model.evaluate(X_test, y_test, verbose=0)
print('Test mse:', score)

Train on 934 samples, validate on 234 samples
Epoch 1/30
934/934 [==============================] - 1s 1ms/sample - loss: 68.3630 - val_loss: 7.2614
Epoch 2/30
934/934 [==============================] - 0s 371us/sample - loss: 6.1735 - val_loss: 3.3769
Epoch 3/30
934/934 [==============================] - 0s 362us/sample - loss: 2.7936 - val_loss: 1.3744
Epoch 4/30
934/934 [==============================] - 0s 394us/sample - loss: 1.2853 - val_loss: 0.6989
Epoch 5/30
934/934 [==============================] - 0s 391us/sample - loss: 0.6654 - val_loss: 0.3530
Epoch 6/30
934/934 [==============================] - 0s 365us/sample - loss: 0.3446 - val_loss: 0.1973
Epoch 7/30
934/934 [==============================] - 0s 358us/sample - loss: 0.1941 - val_loss: 0.1180
Epoch 8/30
934/934 [==============================] - 0s 364us/sample - loss: 0.1165 - val_loss: 0.0867
Epoch 9/30
934/934 [==============================] - 0s 367us/sample - loss: 0.0866 - val_loss: 0.0846
Epoch 10/30
934/934

## 【問題6】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [125]:
# MNISTのデータセットを読み込み
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape) # (60000, 28, 28)
print(X_test.shape) # (10000, 28, 28)
print(X_train[0].dtype) # uint8
print(X_train[0])

(60000, 28, 28)
(10000, 28, 28)
uint8
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0 

In [126]:
# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [127]:
# 平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
print(X_train.shape) # (60000, 784)
print(X_test.shape) # (10000, 784)

(60000, 784)
(10000, 784)


In [128]:
# ワンホット処理
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.fit_transform(y_test[:, np.newaxis])
print(y_train_one_hot.shape) # (60000, 10)
print(y_test_one_hot.shape) # (10000, 10)

(60000, 10)
(10000, 10)


In [129]:
# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)
print('X_train.shape', X_train.shape)
print('X_val.shape', X_val.shape)
print('y_train', y_train.shape)
print('y_val.shape', y_val.shape)

X_train.shape (48000, 784)
X_val.shape (12000, 784)
y_train (48000, 10)
y_val.shape (12000, 10)


In [130]:
K.clear_session()

In [131]:
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 30

# モデル構築
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [133]:
# モデルコンパイル
model.compile(loss='categorical_crossentropy',
             optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
             metrics=['accuracy'])

# 学習
history = model.fit(X_train, y_train,
                   batch_size=batch_size,
                   epochs=num_epochs,
                   verbose=1,
                   validation_data=(X_val,y_val))

# 推定結果(確率)
y_pred_proba = model.predict(X_train)
# 確率を一番高い値のインデックスに変換
y_pred = np.argmax(y_pred_proba, axis=1)

Train on 48000 samples, validate on 12000 samples
Epoch 1/30
48000/48000 [==============================] - 40s 828us/sample - loss: 0.0141 - accuracy: 0.9968 - val_loss: 0.2138 - val_accuracy: 0.9797
Epoch 2/30
48000/48000 [==============================] - 40s 825us/sample - loss: 0.0115 - accuracy: 0.9973 - val_loss: 0.2391 - val_accuracy: 0.9785
Epoch 3/30
48000/48000 [==============================] - 38s 791us/sample - loss: 0.0135 - accuracy: 0.9967 - val_loss: 0.2633 - val_accuracy: 0.9764
Epoch 4/30
48000/48000 [==============================] - 38s 783us/sample - loss: 0.0144 - accuracy: 0.9969 - val_loss: 0.2245 - val_accuracy: 0.9784
Epoch 5/30
48000/48000 [==============================] - 37s 779us/sample - loss: 0.0132 - accuracy: 0.9969 - val_loss: 0.2210 - val_accuracy: 0.9778
Epoch 6/30
48000/48000 [==============================] - 39s 809us/sample - loss: 0.0117 - accuracy: 0.9975 - val_loss: 0.2433 - val_accuracy: 0.9783
Epoch 7/30
48000/48000 [====================

In [134]:
# 指標値計算
score = model.evaluate(X_test, y_test_one_hot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3251868995354289
Test accuracy: 0.9794
